In [1]:
import random
import math
import matplotlib.pyplot as plt
import numpy as np

In [3]:
bounds = np.array([-1, 2]) 
min_bound, max_bound = bounds
min_bound, max_bound

-1

In [13]:
# 创建一个numpy数组
arr = np.array([1, 2, -3, 4, -5, 6])

# 找到所有正数的下标
positive_indices = np.where(arr > 0)[0]
print(positive_indices)  # 输出结果为 [0 1 3 5]

# 将负数替换为0
arr = np.where(arr < 0, 0, arr)
print(arr)  # 输出结果为 [1 2 0 4 0 6]


[0 1 3 5]
[1 2 0 4 0 6]


In [14]:
import random
import math
import matplotlib.pyplot as plt
import numpy as np

# 初始化种群
def generate_initial_population(population_size, num_bits):
    population = []
    for i in range(population_size):
        temporary = []
        # 染色体暂存器
        for j in range(num_bits):
            temporary.append(random.choice([0, 1]))
            # 随机选择一个值，可以是0或1，将其添加到染色体中
        population.append(temporary)
        # 将染色体添加到种群中
    return population


# 从二进制编码转换为十进制数值
def binary_to_decimal(population, bounds):
    decimal_population = []  # 存储所有染色体的十进制数值
    for chromosome in population:  # 遍历种群中的每个染色体
        decimal = 0  # 初始化十进制数值
        for i, gene in enumerate(chromosome):  # 遍历染色体中的每个基因
            decimal += gene * (2 ** i)  # 将基因的值乘以2的幂次方，求和得到十进制数值
        lower_bound, upper_bound = bounds[0], bounds[1]
        mapped_decimal = lower_bound + (decimal / ((2 ** len(chromosome)) - 1)) * (upper_bound - lower_bound)
        decimal_population.append(mapped_decimal)  # 将映射后的十进制数值添加到列表中
    return decimal_population  # 返回所有染色体映射后的十进制数值列表


# 十进制数值转换为二进制编码
def decimal_to_binary(decimal_value, num_bits, bounds):
    # 将十进制数值映射到整数范围
    min_bound, max_bound = bounds
    fixed_point_value = int((decimal_value - min_bound) / (max_bound - min_bound) * (2**num_bits - 1))

    # 将整数转换为二进制编码列表
    binary_value = [int(bit) for bit in format(fixed_point_value, f'0{num_bits}b')]

    return binary_value



# 适应度函数
def fitness_function(x):
    # 计算目标函数值
    y = x * np.sin(10 * math.pi * x) + 2.0
    # 返回适应度（即目标函数值）
    return y


# 适应度分数
def compute_fitness(decimal_population):
    fitness_values = []
    for decimal in decimal_population:
        y = fitness_function(decimal)
        fitness_values.append(y)
    return fitness_values


# 选择操作
def selection(population, fitness_values, num_parents):
    # 保留适应度非负的个体,where返回的是一个二维数组
    positive_fitness_indices = np.where(np.array(fitness_values) >= 0)[0]
    # 根据下标找出个体和他的适应度值
    population = [population[i] for i in positive_fitness_indices]
    fitness_values = [fitness_values[i] for i in positive_fitness_indices]

    # 计算适应度总和
    fitness_sum = sum(fitness_values)

    # 计算每个个体的选择概率，与适应度分数成正比
    probabilities = [fitness_value / fitness_sum for fitness_value in fitness_values]

    # 计算累积概率分布
    cumulative_probabilities = np.cumsum(probabilities)

    # 选择父代个体
    parents = []
    for i in range(num_parents):
        # 产生一个0到1之间的随机数
        rand_num = np.random.uniform(low=0, high=1.0)
        # random.random()
        # 确定随机数出现在哪个个体的概率区域内
        for j in range(len(cumulative_probabilities)):
            # 当前随机数小于等于累积概率列表中的某个元素，就选择该元素对应的个体作为父代
            if rand_num <= cumulative_probabilities[j]:
                parents.append(population[j])  # 直接返回基因
                break

    return parents


def single_point_crossover(parents, crossover_rate):
    offspring = []
    num_parents = len(parents)
    num_bits = len(parents[0])

    for i in range(0, num_parents - 1, 2):
        parent1 = parents[i]
        parent2 = parents[i + 1]

        # 随机生成交叉点
        crossover_point = np.random.randint(1, num_bits)

        # 根据交叉率进行交叉操作
        if np.random.random() < crossover_rate:
            # 生成新的后代
            offspring1 = parent1[:crossover_point] + parent2[crossover_point:]
            offspring2 = parent2[:crossover_point] + parent1[crossover_point:]
        else:
            # 如果不交叉，则直接将父代作为后代
            offspring1 = parent1
            offspring2 = parent2

        # 将后代添加到列表中
        offspring.append(offspring1)
        offspring.append(offspring2)

    return offspring


# 变异操作
def mutation(offspring, mutation_rate):
    # 遍历每个后代
    for i in range(len(offspring)):
        # 遍历每个后代的基因
        for j in range(len(offspring[i])):
            # 判断是否进行变异操作
            if np.random.uniform(0, 1) <= mutation_rate:
                # 随机将基因进行变异
                offspring[i][j] = 1 - offspring[i][j]
    # 返回变异后的后代
    return offspring


In [95]:
population_size = 10
num_generations = 500
num_parents = 10
num_bits = 10  # 每个个体包含的基因数量
crossover_rate = 0.9  # 交叉率
mutation_rate = 0.1  # 变异率
bounds = np.array([-1, 2])  # 搜索空间边界，即每个基因x的取值范围为[-5, 5]

In [96]:
# 初始化种群
population = generate_initial_population(population_size, num_bits)
len(population[0])
population

[[0, 0, 1, 1, 0, 0, 0, 0, 1, 0],
 [1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
 [0, 1, 0, 0, 1, 1, 0, 0, 1, 1],
 [1, 1, 1, 0, 1, 0, 0, 0, 0, 1],
 [1, 1, 0, 0, 1, 1, 0, 1, 1, 1],
 [1, 0, 1, 1, 0, 0, 0, 1, 0, 0],
 [1, 0, 1, 1, 1, 1, 0, 1, 1, 0],
 [1, 0, 0, 0, 0, 1, 1, 0, 1, 1],
 [1, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]]

In [79]:
import copy
population_copy = copy.deepcopy(population)
population_copy

[[1, 1, 0, 0, 1, 1, 1, 1, 1, 0],
 [1, 1, 0, 1, 1, 0, 1, 1, 0, 0],
 [1, 0, 1, 1, 0, 0, 0, 1, 1, 0],
 [1, 0, 1, 0, 0, 0, 0, 0, 1, 1],
 [0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
 [0, 1, 1, 0, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 0, 0, 1, 0, 1, 0, 1],
 [0, 0, 1, 1, 0, 0, 1, 0, 1, 0]]

In [97]:
# 迭代num_generations轮
best_fitness = float('-inf')
best_individuals = []
best_fitnesses = []

In [98]:
# 二进制转换为十进制
decimal_population = binary_to_decimal(population, bounds)
# 计算适应度分数
fitness_values = compute_fitness(decimal_population)
# 选择父代个体
parents = selection(population, fitness_values, num_parents)
# 交叉操作
offspring = single_point_crossover(parents, crossover_rate)


In [99]:
index_max=fitness_values.index(max(fitness_values))
local_optimum=population[index_max]
local_optimum

[1, 0, 0, 0, 0, 0, 1, 1, 1, 1]

In [100]:
decimal_population

[-0.21407624633431088,
 0.6510263929618767,
 1.3988269794721404,
 0.5689149560117301,
 1.7771260997067446,
 -0.5865102639296188,
 0.3049853372434017,
 1.5366568914956011,
 1.8181818181818183,
 -0.9941348973607038]

In [101]:
fitness_values

[2.091612954455781,
 2.650687971384071,
 1.948462756922927,
 1.528608810542721,
 0.8300555300818924,
 1.7588149827288215,
 1.952428547010304,
 0.5963860825498297,
 2.9829833044647245,
 1.8178578011410145]

In [102]:
parents

[[1, 0, 1, 1, 1, 1, 0, 1, 1, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 1, 1, 0, 0, 1, 1],
 [1, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
 [1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
 [0, 1, 0, 0, 1, 1, 0, 0, 1, 1],
 [1, 0, 1, 1, 1, 1, 0, 1, 1, 0],
 [1, 1, 1, 0, 1, 0, 0, 0, 0, 1],
 [1, 0, 1, 1, 0, 0, 0, 1, 0, 0]]

In [103]:
offspring

[[1, 0, 1, 1, 1, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 1, 1, 0],
 [0, 1, 0, 0, 1, 0, 1, 1, 1, 1],
 [1, 0, 0, 0, 0, 1, 0, 0, 1, 1],
 [1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
 [1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
 [0, 1, 0, 0, 1, 1, 0, 1, 1, 0],
 [1, 0, 1, 1, 1, 1, 0, 0, 1, 1],
 [1, 1, 1, 1, 0, 0, 0, 1, 0, 0],
 [1, 0, 1, 0, 1, 0, 0, 0, 0, 1]]

In [104]:
# 变异操作
offspring = mutation(offspring, mutation_rate)
offspring

[[1, 0, 1, 1, 1, 1, 0, 0, 0, 0],
 [1, 1, 0, 0, 0, 0, 0, 1, 1, 0],
 [0, 1, 0, 0, 0, 0, 1, 1, 1, 1],
 [1, 0, 0, 0, 0, 1, 0, 0, 1, 1],
 [1, 1, 0, 0, 1, 1, 1, 0, 0, 1],
 [1, 1, 0, 0, 1, 1, 1, 0, 0, 1],
 [0, 1, 0, 0, 1, 0, 0, 1, 1, 0],
 [1, 0, 1, 1, 1, 1, 0, 0, 1, 0],
 [1, 1, 1, 1, 0, 0, 0, 1, 0, 0],
 [1, 0, 1, 1, 1, 0, 0, 0, 0, 1]]

In [105]:
decimal_offspring = binary_to_decimal(offspring, bounds)
# 计算适应度分数
fitness_offspring = compute_fitness(decimal_offspring)
fitness_offspring

[2.5055947166787518,
 1.8800036717750574,
 3.1213368537768122,
 0.6517276268618357,
 2.7865017882523913,
 2.7865017882523913,
 1.8898525795806995,
 2.056440074010466,
 1.6682958751062507,
 1.7588149827288198]

In [106]:
index_min=fitness_offspring.index(min(fitness_offspring))
offspring[index_min]=local_optimum
offspring

[[1, 0, 1, 1, 1, 1, 0, 0, 0, 0],
 [1, 1, 0, 0, 0, 0, 0, 1, 1, 0],
 [0, 1, 0, 0, 0, 0, 1, 1, 1, 1],
 [1, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [1, 1, 0, 0, 1, 1, 1, 0, 0, 1],
 [1, 1, 0, 0, 1, 1, 1, 0, 0, 1],
 [0, 1, 0, 0, 1, 0, 0, 1, 1, 0],
 [1, 0, 1, 1, 1, 1, 0, 0, 1, 0],
 [1, 1, 1, 1, 0, 0, 0, 1, 0, 0],
 [1, 0, 1, 1, 1, 0, 0, 0, 0, 1]]

In [107]:
# 得到新的种群
population = offspring

In [108]:
population

[[1, 0, 1, 1, 1, 1, 0, 0, 0, 0],
 [1, 1, 0, 0, 0, 0, 0, 1, 1, 0],
 [0, 1, 0, 0, 0, 0, 1, 1, 1, 1],
 [1, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [1, 1, 0, 0, 1, 1, 1, 0, 0, 1],
 [1, 1, 0, 0, 1, 1, 1, 0, 0, 1],
 [0, 1, 0, 0, 1, 0, 0, 1, 1, 0],
 [1, 0, 1, 1, 1, 1, 0, 0, 1, 0],
 [1, 1, 1, 1, 0, 0, 0, 1, 0, 0],
 [1, 0, 1, 1, 1, 0, 0, 0, 0, 1]]

In [109]:
generation_best_fitness = fitness_values[index_max]
generation_best_fitness

2.9829833044647245

In [110]:
generation_best_individual = decimal_population[index_max]
generation_best_individual

1.8181818181818183

In [131]:
a=[[0,1],[3,5]]
b=a
a[0]=4
b

[4, [3, 5]]

In [1]:
class MyClass:
    class_property = 0

    def __init__(self, instance_property):
        self.instance_property = instance_property
      
    def instance_method(self):
        print('this is an instance method')
      
    @classmethod
    def class_method(cls):
        print('this is a class method')
        print(f'class property: {cls.class_property}')


In [2]:
c1 = MyClass(10)
c2 = MyClass(20)

# 调用类方法
MyClass.class_method() # 输出：this is a class method; class property: 0

# 在类方法中修改类属性
MyClass.class_property = 100

# 输出实例对象的属性值
print(c1.instance_property) # 输出：10
print(c2.instance_property) # 输出：20

# 重新调用类方法
MyClass.class_method() # 输出：this is a class method; class property: 100


this is a class method
class property: 0
10
20
this is a class method
class property: 100


In [3]:
c1.class_method()

this is a class method
class property: 100


In [61]:
class Document:
    _registry = {}
      
    def __init__(self, name):
        self.name = name
      
    def description(self):
        return f'Document: {self.name}'
      
    @classmethod
    def register(cls, name, factory):
        print(factory(name).name)
        cls._registry[name] = lambda: factory(name)
      
    @classmethod
    def create(cls, name):
        if name not in cls._registry:
            raise ValueError(f'{name} is not a registered document type')
        return cls._registry[name]()
      
class Resume(Document):
    def description(self):
        return f'Resume: {self.name}'
      
class Report(Document):
    def description(self):
        return f'Report: {self.name}'
      
Document.register('resume', Resume)
Document.register('report', Report)



resume
report


In [63]:
a1=Document("resume")
a1_class=a1.create("resume")
a1_class.name

'resume'

In [49]:
print(Document._registry)

{'resume': <function Document.register.<locals>.<lambda> at 0x0000020B8D8B7CA8>, 'report': <function Document.register.<locals>.<lambda> at 0x0000020B8D8B79D8>}


In [57]:
a1=Document("a1")
a1_class=a1.create('resume')
a1_class_ga=a1_class.create('report')
a1_class.name,a1_class_ga.name

('resume', 'report')

In [53]:
my_resume1 = Document.create('resume')
my_resume2 = Document.create('report')
my_resume1.description(),my_resume2.description()

('Resume: resume', 'Report: report')